In [1]:
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error
from datetime import datetime, timedelta
import ccxt
import ta
from  pyearth import Earth
import plotly.graph_objects as go
from sklearn import preprocessing
import re
from tqdm import tqdm

In [2]:
def get_node_mars(model):
    list_summary = model.summary().split('\n')
    new_list_summary = []
                
    for j in list_summary:
        if 'Yes' not in j:
            new_list_summary.append(j)

    basis_function_numbers = []

    for line in new_list_summary:
        match = re.search(r'h\(x0-([\d.]+)\)|h\(([\d.]+)-x0\)|h\(([\d.]+)\)\s+No', line)
        if match:
            basis_function_numbers.extend(match.group(1, 2, 3))

    basis_number = []
    for i in basis_function_numbers:
        if not i is None:
            basis_number.append(float(i))
    basis_number.sort()
    return basis_number

In [3]:
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

def execute_query(connection, query, varlist=None):
    cursor = connection.cursor(buffered=True)
    try:
        if varlist is None:
            cursor.execute(query)
        else:
            cursor.execute(query, varlist)
        connection.commit()
        return cursor.fetchall()
    except Error as err:
        print(f"Error: '{err}'")

def executemany_query(connection, query, varlist=None):
    cursor = connection.cursor()
    try:
        if varlist is None:
            cursor.executemany(query)
        else:
            cursor.executemany(query, varlist)
        connection.commit()
    except Error as err:
        print(f"Error: '{err}'")

In [4]:
pw = '27111998'
db = 'crypto_db'
connection = create_db_connection("localhost", "root", pw, db)

MySQL Database connection successful


In [5]:
exchange_id = 'tokocrypto'
exchange_class = getattr(ccxt, exchange_id)
exchange = exchange_class({
    'apiKey': 'YOUR_API_KEY',
    'secret': 'YOUR_SECRET',
})
markets = exchange.load_markets()
market_list = list(markets.keys())
considered_market = []
for i in market_list:
    if '/USDT' in i or '/BTC' in i:
        considered_market.append(i)

print(len(considered_market))
print(considered_market)

606
['1INCH/BTC', '1INCH/USDT', 'AAVE/USDT', 'ACA/BTC', 'ACA/USDT', 'ACE/BTC', 'ACE/USDT', 'ACH/BTC', 'ACH/USDT', 'ACM/USDT', 'ADA/BTC', 'ADA/USDT', 'ADX/BTC', 'ADX/USDT', 'AERGO/BTC', 'AERGO/USDT', 'AGIX/BTC', 'AGIX/USDT', 'AGLD/BTC', 'AGLD/USDT', 'AI/BTC', 'AI/USDT', 'AKRO/USDT', 'ALCX/BTC', 'ALCX/USDT', 'ALGO/BTC', 'ALGO/USDT', 'ALICE/BTC', 'ALICE/USDT', 'ALPACA/BTC', 'ALPACA/USDT', 'ALPHA/BTC', 'ALPHA/USDT', 'ALPINE/BTC', 'ALPINE/USDT', 'ALT/BTC', 'ALT/USDT', 'AMP/USDT', 'ANKR/BTC', 'ANKR/USDT', 'APE/BTC', 'APE/USDT', 'API3/BTC', 'API3/USDT', 'APT/BTC', 'APT/USDT', 'ARB/BTC', 'ARB/USDT', 'ARKM/BTC', 'ARKM/USDT', 'ARPA/BTC', 'ARPA/USDT', 'AR/BTC', 'AR/USDT', 'ASR/USDT', 'ASTR/USDT', 'AST/BTC', 'ATA/BTC', 'ATA/USDT', 'ATM/USDT', 'ATOM/BTC', 'ATOM/USDT', 'AUCTION/BTC', 'AUCTION/USDT', 'AUDIO/BTC', 'AUDIO/USDT', 'AVAX/BTC', 'AVAX/USDT', 'AVA/BTC', 'AVA/USDT', 'AXS/BTC', 'AXS/USDT', 'BADGER/BTC', 'BADGER/USDT', 'BAL/BTC', 'BAL/USDT', 'BAND/BTC', 'BAND/USDT', 'BAR/USDT', 'BAT/BTC', 'BAT/

In [6]:
data_pump = pd.read_csv("data_pump.csv")
data_pump

,Unnamed: 0,market,start,base_price,end,last_price,increase,pump_length,date_listing,pump_to_listing
0,0,1INCHBTC,2021-01-08,0.000031,2021-02-05,0.000152,3.881960,28 days,2020-12-25,14 days
1,1,1INCHUSDT,2021-01-02,1.062220,2021-02-05,5.825000,4.483798,34 days,2020-12-25,8 days
2,2,AAVEUSDT,2020-11-04,28.570800,2020-12-05,92.714000,2.245061,31 days,2020-10-15,20 days
3,3,AAVEUSDT,2020-12-26,76.131200,2021-02-12,528.325000,5.939665,48 days,2020-10-15,72 days
4,4,ACAUSDT,2023-06-10,0.032340,2023-07-12,0.075700,1.340754,32 days,2022-01-25,501 days
...,...,...,...,...,...,...,...,...,...,...
1241,1241,ZRXBTC,2017-12-11,0.000015,2018-01-27,0.000184,11.382234,47 days,2017-09-13,89 days
1242,1242,ZRXBTC,2021-01-21,0.000016,2021-02-05,0.000041,1.632614,15 days,2017-09-13,1226 days
1243,1243,ZRXUSDT,2020-04-23,0.184960,2020-05-09,0.422600,1.284818,16 days,2019-02-28,420 days
1244,1244,ZRXUSDT,2021-01-21,0.514640,2021-02-13,1.891200,2.674802,23 days,2019-02-28,693 days


In [7]:
data_pump['pump_to_listing'] = pd.to_timedelta(data_pump['pump_to_listing'])

In [8]:
considered_data = data_pump[data_pump['pump_to_listing'] > timedelta(days=30)]
considered_data

,Unnamed: 0,market,start,base_price,end,last_price,increase,pump_length,date_listing,pump_to_listing
3,3,AAVEUSDT,2020-12-26,7.613120e+01,2021-02-12,528.325000,5.939665,48 days,2020-10-15,72 days
4,4,ACAUSDT,2023-06-10,3.234000e-02,2023-07-12,0.075700,1.340754,32 days,2022-01-25,501 days
5,5,ACHBTC,2023-01-24,4.800000e-07,2023-02-23,0.000002,2.979167,30 days,2022-01-10,379 days
6,6,ACHUSDT,2023-01-24,1.090800e-02,2023-02-23,0.045660,3.185919,30 days,2022-01-10,379 days
8,8,ADABTC,2021-01-29,1.028400e-05,2021-02-28,0.000029,1.828666,30 days,2017-11-30,1156 days
...,...,...,...,...,...,...,...,...,...,...
1241,1241,ZRXBTC,2017-12-11,1.486000e-05,2018-01-27,0.000184,11.382234,47 days,2017-09-13,89 days
1242,1242,ZRXBTC,2021-01-21,1.576000e-05,2021-02-05,0.000041,1.632614,15 days,2017-09-13,1226 days
1243,1243,ZRXUSDT,2020-04-23,1.849600e-01,2020-05-09,0.422600,1.284818,16 days,2019-02-28,420 days
1244,1244,ZRXUSDT,2021-01-21,5.146400e-01,2021-02-13,1.891200,2.674802,23 days,2019-02-28,693 days


In [33]:
idx = 3
market = considered_data.iloc[idx]['market']
start_pump = considered_data.iloc[idx]['start']
query = 'SELECT * FROM %s ORDER BY timestamp ASC' % market
res = execute_query(connection,query)
df = pd.DataFrame(res,columns=['timestamp', 'open','high', 'low','close','volume'])
df = df.drop_duplicates()
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df = df[df['timestamp'] <= start_pump]
day_df = (df.resample('D', on='timestamp').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last','volume': 'mean'}))
day_df['timestamp'] = day_df.index
num_idx = min([365, len(day_df)])

model_price = Earth()
scaler = preprocessing.MinMaxScaler(feature_range=(0,100))
scaled_price = scaler.fit_transform(day_df['close'].iloc[-num_idx:].values.reshape(-1,1))
# model_price.fit(day_df['timestamp'].iloc[-365:]-day_df['timestamp'].iloc[365],day_df['close'].iloc[-365:])
# model_price.fit(day_df['timestamp'].iloc[-365:]-day_df['timestamp'].iloc[365],scaled_price.flatten())

arr_x = np.arange(num_idx)
model_price.fit(arr_x,scaled_price.flatten())

e:\Harry\crypto\.conda\lib\site-packages\pyearth\earth.py:813: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.

e:\Harry\crypto\.conda\lib\site-packages\pyearth\earth.py:1066: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.



Earth(allow_linear=None, allow_missing=False, check_every=None,
      enable_pruning=True, endspan=None, endspan_alpha=None, fast_K=None,
      fast_h=None, feature_importance_type=None, max_degree=None,
      max_terms=None, min_search_points=None, minspan=None, minspan_alpha=None,
      penalty=None, smooth=None, thresh=None, use_fast=None, verbose=0,
      zero_tol=None)

In [10]:
# y_pred = model_price.predict(day_df['timestamp'].iloc[-365:]-day_df['timestamp'].iloc[365])
y_pred = model_price.predict(arr_x)
basis_node = get_node_mars(model_price)
last_node = int(basis_node[-1])
last_slope = (y_pred[-1] - y_pred[last_node])/(arr_x[-1] - arr_x[last_node])

fig = go.Figure()
# fig.add_trace(go.Scatter(x=day_df['timestamp'].iloc[-365:],y=scaled_price.flatten()))
# fig.add_trace(go.Scatter(x=day_df['timestamp'].iloc[-365:],y=y_pred,mode='lines'))

fig.add_trace(go.Scatter(x=np.arange(365),y=scaled_price.flatten()))
fig.add_trace(go.Scatter(x=np.arange(365),y=y_pred,mode='lines'))

fig.show()
print(last_slope)

0.40721954682094746


In [11]:
adi = ta.volume.AccDistIndexIndicator(high=df['high'],low=df['low'],close=df['close'],volume=df['volume']).acc_dist_index()
model_adi = Earth()
scaler_adi = preprocessing.MinMaxScaler(feature_range=(0,1000))
data_num_adi = min([2000,len(df)])
scaled_adi = scaler_adi.fit_transform(df['close'].iloc[-data_num_adi:].values.reshape(-1,1))
# model_price.fit(day_df['timestamp'].iloc[-365:]-day_df['timestamp'].iloc[365],day_df['close'].iloc[-365:])
# model_price.fit(day_df['timestamp'].iloc[-365:]-day_df['timestamp'].iloc[365],scaled_price.flatten())
arr_x_adi = np.arange(data_num_adi)/24
model_adi.fit(arr_x_adi,scaled_adi.flatten())

# y_pred = model_price.predict(day_df['timestamp'].iloc[-365:]-day_df['timestamp'].iloc[365])
y_pred_adi = model_adi.predict(arr_x_adi)
basis_node = get_node_mars(model_adi)
last_node = basis_node[-1]
last_slope_adi = (y_pred_adi[-1] - model_adi.predict([last_node]))/(arr_x[-1] - last_node)

fig = go.Figure()
# fig.add_trace(go.Scatter(x=day_df['timestamp'].iloc[-365:],y=scaled_price.flatten()))
# fig.add_trace(go.Scatter(x=day_df['timestamp'].iloc[-365:],y=y_pred,mode='lines'))

fig.add_trace(go.Scatter(x=arr_x_adi,y=scaled_adi.flatten()))
fig.add_trace(go.Scatter(x=arr_x_adi,y=y_pred_adi,mode='lines'))

fig.show()
print(last_slope_adi)

e:\Harry\crypto\.conda\lib\site-packages\pyearth\earth.py:813: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.

e:\Harry\crypto\.conda\lib\site-packages\pyearth\earth.py:1066: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.



[-0.17945139]


In [12]:
aroon = ta.trend.AroonIndicator(low=day_df['low'],high=day_df['high'])
aroon_up = aroon.aroon_up()
aroon_down = aroon.aroon_down()
aroon_diff = aroon.aroon_indicator()

In [13]:
macd = ta.trend.MACD(close=pd.Series(scaled_price.flatten()))
macd_diff = macd.macd_diff()
# scaler = preprocessing.MinMaxScaler(feature_range=(-1,1))
# macd_diff = scaler.fit_transform(macd_diff.values.reshape(-1,1))
macd_line = macd.macd()
macd_signal = macd.macd_signal()

In [16]:
stoch_oscillator = ta.momentum.StochasticOscillator(high=day_df['high'], low=day_df['low'], close=day_df['close'])
stochastic = stoch_oscillator.stoch()
stochastic_signal = stoch_oscillator.stoch_signal()
delta_stochastic = stochastic - stochastic_signal

In [17]:
rsi_indicator = ta.momentum.RSIIndicator(close=pd.Series(scaled_price.flatten()))
rsi_val = rsi_indicator.rsi()

In [131]:
start_pump = not_pump['start_date'].iloc[-2]
num_idx = 365

In [132]:
query = 'SELECT * FROM btcusdt ORDER BY timestamp ASC'
res = execute_query(connection,query)
df_btc = pd.DataFrame(res,columns=['timestamp', 'open','high', 'low','close','volume'])
df_btc = df_btc.drop_duplicates()
df_btc['timestamp'] = pd.to_datetime(df_btc['timestamp'], unit='ms')

df_btc = df_btc[df_btc['timestamp'] <= start_pump]
day_df_btc = (df_btc.resample('D', on='timestamp').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last','volume': 'mean'}))
day_df_btc['timestamp'] = day_df_btc.index

alt_vs_btc = pd.DataFrame({'alt': day_df['close'].iloc[-num_idx:], 'btc': day_df_btc['close'].iloc[-num_idx:]})
df_corr = alt_vs_btc.corr()
coef_corr = df_corr['alt']['btc']

model_btc = Earth()
scaler_btc = preprocessing.MinMaxScaler(feature_range=(0,100))
scaled_price_btc = scaler_btc.fit_transform(day_df_btc['close'].iloc[-num_idx:].values.reshape(-1,1))

arr_x = np.arange(num_idx)
model_btc.fit(arr_x,scaled_price_btc.flatten())

y_pred_btc = model_btc.predict(arr_x)
basis_node_btc = get_node_mars(model_btc)
last_node_btc = int(basis_node_btc[-1])
last_slope_btc = (y_pred_btc[-1] - y_pred_btc[last_node_btc])/(arr_x[-1] - arr_x[last_node_btc])

fig = go.Figure()
# fig.add_trace(go.Scatter(x=day_df['timestamp'].iloc[-365:],y=scaled_price.flatten()))
# fig.add_trace(go.Scatter(x=day_df['timestamp'].iloc[-365:],y=y_pred,mode='lines'))

fig.add_trace(go.Scatter(x=np.arange(365),y=scaled_price_btc.flatten()))
fig.add_trace(go.Scatter(x=np.arange(365),y=y_pred_btc,mode='lines'))

fig.show()

print(last_slope_btc)

-0.3077735978699435


GENERATE DATA

In [69]:
import warnings
warnings.filterwarnings('ignore')

arr_last_slope = []
arr_last_slope_adi = []
arr_aroon_up = []
arr_aroon_down = []
arr_aroon_diff = []
arr_macd_diff = []
arr_macd_line = []
arr_macd_signal = []
arr_stochastic = []
arr_stochastic_signal = []
arr_delta_stochastic = []
arr_rsi_val = []
arr_coef_corr = []
arr_last_slope_btc = []

for idx in tqdm(range(len(considered_data))):
    try:
        market = considered_data.iloc[idx]['market']
        start_pump = considered_data.iloc[idx]['start']
        query = 'SELECT * FROM %s ORDER BY timestamp ASC' % market
        res = execute_query(connection,query)
        df = pd.DataFrame(res,columns=['timestamp', 'open','high', 'low','close','volume'])
        df = df.drop_duplicates()
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df = df[df['timestamp'] <= start_pump]
        day_df = (df.resample('D', on='timestamp').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last','volume': 'mean'}))
        day_df['timestamp'] = day_df.index
        num_idx = min([365, len(day_df)])

        model_price = Earth()
        scaler = preprocessing.MinMaxScaler(feature_range=(0,100))
        scaled_price = scaler.fit_transform(day_df['close'].iloc[-num_idx:].values.reshape(-1,1))

        arr_x = np.arange(num_idx)
        model_price.fit(arr_x,scaled_price.flatten())
        y_pred = model_price.predict(arr_x)
        basis_node = get_node_mars(model_price)
        last_node = int(basis_node[-1])
        last_slope = (y_pred[-1] - y_pred[last_node])/(arr_x[-1] - arr_x[last_node])

        adi = ta.volume.AccDistIndexIndicator(high=df['high'],low=df['low'],close=df['close'],volume=df['volume']).acc_dist_index()
        model_adi = Earth()
        scaler_adi = preprocessing.MinMaxScaler(feature_range=(0,1000))
        data_num_adi = min([2000,len(df)])
        scaled_adi = scaler_adi.fit_transform(df['close'].iloc[-data_num_adi:].values.reshape(-1,1))
        arr_x_adi = np.arange(data_num_adi)/24
        model_adi.fit(arr_x_adi,scaled_adi.flatten())

        y_pred_adi = model_adi.predict(arr_x_adi)
        basis_node = get_node_mars(model_adi)
        last_node = basis_node[-1]
        last_slope_adi = (y_pred_adi[-1] - model_adi.predict([last_node]))[0]/(arr_x[-1] - last_node)

        aroon = ta.trend.AroonIndicator(low=day_df['low'],high=day_df['high'])
        aroon_up = aroon.aroon_up()[-1]
        aroon_down = aroon.aroon_down()[-1]
        aroon_diff = aroon.aroon_indicator()[-1]

        macd = ta.trend.MACD(close=pd.Series(scaled_price.flatten()))
        macd_diff = macd.macd_diff().values[-1]
        macd_line = macd.macd().values[-1]
        macd_signal = macd.macd_signal().values[-1]

        stoch_oscillator = ta.momentum.StochasticOscillator(high=day_df['high'], low=day_df['low'], close=day_df['close'])
        stochastic = stoch_oscillator.stoch().values[-1]
        stochastic_signal = stoch_oscillator.stoch_signal().values[-1]
        delta_stochastic = stochastic - stochastic_signal

        rsi_indicator = ta.momentum.RSIIndicator(close=pd.Series(scaled_price.flatten()))
        rsi_val = rsi_indicator.rsi().values[-1]

        query = 'SELECT * FROM btcusdt ORDER BY timestamp ASC'
        res = execute_query(connection,query)
        df_btc = pd.DataFrame(res,columns=['timestamp', 'open','high', 'low','close','volume'])
        df_btc = df_btc.drop_duplicates()
        df_btc['timestamp'] = pd.to_datetime(df_btc['timestamp'], unit='ms')
        df_btc = df_btc[df_btc['timestamp'] <= start_pump]
        
        day_df_btc = (df_btc.resample('D', on='timestamp').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last','volume': 'mean'}))
        day_df_btc['timestamp'] = day_df_btc.index

        alt_vs_btc = pd.DataFrame({'alt': day_df['close'].iloc[-num_idx:], 'btc': day_df_btc['close'].iloc[-num_idx:]})
        df_corr = alt_vs_btc.corr()
        coef_corr = df_corr['alt']['btc']

        model_btc = Earth()
        scaler_btc = preprocessing.MinMaxScaler(feature_range=(0,100))
        scaled_price_btc = scaler_btc.fit_transform(day_df_btc['close'].iloc[-num_idx:].values.reshape(-1,1))

        arr_x = np.arange(num_idx)
        model_btc.fit(arr_x,scaled_price_btc.flatten())

        y_pred_btc = model_btc.predict(arr_x)
        basis_node_btc = get_node_mars(model_btc)
        last_node_btc = int(basis_node_btc[-1])
        last_slope_btc = (y_pred_btc[-1] - y_pred_btc[last_node_btc])/(arr_x[-1] - arr_x[last_node_btc])

        arr_last_slope.append(last_slope)
        arr_last_slope_adi.append(last_slope_adi)
        arr_aroon_up.append(aroon_up)
        arr_aroon_down.append(aroon_down)
        arr_aroon_diff.append(aroon_diff)
        arr_macd_diff.append(macd_diff)
        arr_macd_line.append(macd_line)
        arr_macd_signal.append(macd_signal)
        arr_stochastic.append(stochastic)
        arr_stochastic_signal.append(stochastic_signal)
        arr_delta_stochastic.append(delta_stochastic)
        arr_rsi_val.append(rsi_val)
        arr_coef_corr.append(coef_corr)
        arr_last_slope_btc.append(last_slope_btc)
    except Exception as e:
        print('Error: ', e)
        arr_last_slope.append(np.nan)
        arr_last_slope_adi.append(np.nan)
        arr_aroon_up.append(np.nan)
        arr_aroon_down.append(np.nan)
        arr_aroon_diff.append(np.nan)
        arr_macd_diff.append(np.nan)
        arr_macd_line.append(np.nan)
        arr_macd_signal.append(np.nan)
        arr_stochastic.append(np.nan)
        arr_stochastic_signal.append(np.nan)
        arr_delta_stochastic.append(np.nan)
        arr_rsi_val.append(np.nan)
        arr_coef_corr.append(np.nan)
        arr_last_slope_btc.append(np.nan)
        continue

 14%|█▍        | 165/1143 [18:23<1:15:03,  4.61s/it]

Error:  X and y do not have compatible dimensions.


 15%|█▍        | 166/1143 [18:25<1:06:22,  4.08s/it]

Error:  X and y do not have compatible dimensions.


 15%|█▍        | 167/1143 [18:29<1:02:49,  3.86s/it]

...

 70%|███████   | 803/1143 [1:27:40<36:59,  6.53s/it]  

Error:  list index out of range


 89%|████████▊ | 1013/1143 [1:51:23<07:18,  3.37s/it] 

Error:  list index out of range


100%|██████████| 1143/1143 [2:04:39<00:00,  6.54s/it]


In [92]:
considered_data['last_slope'] = arr_last_slope
arr_last_slope_adi_rev = []
for i in arr_last_slope_adi:
    if not np.isnan(i):
        arr_last_slope_adi_rev.append(i[0])
    else:
        arr_last_slope_adi_rev.append(i)
considered_data['last_slope_adi'] = arr_last_slope_adi_rev
considered_data['aroon_up'] = arr_aroon_up
considered_data['aroon_down'] = arr_aroon_down
considered_data['aroon_diff'] = arr_aroon_diff
considered_data['macd_diff'] = arr_macd_diff
considered_data['macd_line'] = arr_macd_line
considered_data['macd_signal'] = arr_macd_signal
considered_data['stochastic'] = arr_stochastic
considered_data['stochastic_signal'] = arr_stochastic_signal
considered_data['delta_stochastic'] = arr_delta_stochastic
considered_data['rsi_val'] = arr_rsi_val
considered_data['coef_corr'] = arr_coef_corr
considered_data['last_slope_btc'] = arr_last_slope_btc


In [93]:
considered_data_dropna = considered_data.dropna()
considered_data_dropna

,Unnamed: 0,market,start,base_price,end,last_price,increase,pump_length,date_listing,pump_to_listing,...,aroon_diff,macd_diff,macd_line,macd_signal,stochastic,stochastic_signal,delta_stochastic,rsi_val,coef_corr,last_slope_btc
3,3,AAVEUSDT,2020-12-26,7.613120e+01,2021-02-12,528.325000,5.939665,48 days,2020-10-15,72 days,...,-16.0,-2.521783,2.580118,5.101901,47.739282,50.220167,-2.480885,49.144531,0.854646,0.610359
4,4,ACAUSDT,2023-06-10,3.234000e-02,2023-07-12,0.075700,1.340754,32 days,2022-01-25,501 days,...,-76.0,0.006086,-2.137793,-2.143879,6.565657,11.279461,-4.713805,31.384615,-0.288436,-0.380426
5,5,ACHBTC,2023-01-24,4.800000e-07,2023-02-23,0.000002,2.979167,30 days,2022-01-10,379 days,...,8.0,-0.035125,0.167486,0.202612,20.000000,21.111111,-1.111111,48.166937,0.794324,1.307858
6,6,ACHUSDT,2023-01-24,1.090800e-02,2023-02-23,0.045660,3.185919,30 days,2022-01-10,379 days,...,68.0,0.240458,1.482955,1.242497,42.995839,42.163662,0.832178,59.783568,0.935018,1.307858
8,8,ADABTC,2021-01-29,1.028400e-05,2021-02-28,0.000029,1.828666,30 days,2017-11-30,1156 days,...,80.0,-0.483199,7.123728,7.606927,68.932039,70.820360,-1.888321,59.492533,0.138943,0.021751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1241,1241,ZRXBTC,2017-12-11,1.486000e-05,2018-01-27,0.000184,11.382234,47 days,2017-09-13,89 days,...,-52.0,-0.711679,-7.256399,-6.544719,15.224064,19.684878,-4.460814,28.476598,-0.853006,4.531867
1242,1242,ZRXBTC,2021-01-21,1.576000e-05,2021-02-05,0.000041,1.632614,15 days,2017-09-13,1226 days,...,36.0,1.064627,0.157795,-0.906832,72.173913,71.400966,0.772947,54.073124,-0.545476,-0.892777
1243,1243,ZRXUSDT,2020-04-23,1.849600e-01,2020-05-09,0.422600,1.284818,16 days,2019-02-28,420 days,...,48.0,0.407085,-0.071018,-0.478104,36.826347,34.231537,2.594810,50.311997,0.187440,0.239221
1244,1244,ZRXUSDT,2021-01-21,5.146400e-01,2021-02-13,1.891200,2.674802,23 days,2019-02-28,693 days,...,76.0,0.978360,7.220374,6.242013,66.105769,67.120726,-1.014957,59.885908,0.461472,-0.892777


In [135]:
print(considered_data['last_slope_btc'].values)

[ 0.61035906 -0.38042633  1.30785825 ...  0.23922111 -0.89277702
  2.93903989]


In [94]:
considered_data_dropna.to_csv('data_pump_not_ico.csv')

In [7]:
import random
import warnings
warnings.filterwarnings('ignore')

arr_last_slope = []
arr_last_slope_adi = []
arr_aroon_up = []
arr_aroon_down = []
arr_aroon_diff = []
arr_macd_diff = []
arr_macd_line = []
arr_macd_signal = []
arr_stochastic = []
arr_stochastic_signal = []
arr_delta_stochastic = []
arr_rsi_val = []
arr_coef_corr = []
arr_last_slope_btc = []


arr_is_pump = []

arr_market = []
arr_start_date = []

WINDOW_AFTER = 14
for idx in tqdm(range(1000)):
    try:
        market = random.choice(considered_market)
        market = market.replace('/','')
        query = 'SELECT * FROM %s ORDER BY timestamp ASC' % market
        res = execute_query(connection,query)
        df = pd.DataFrame(res,columns=['timestamp', 'open','high', 'low','close','volume'])
        df = df.drop_duplicates()
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        # df = df[df['timestamp'] <= start_pump]
        day_df = (df.resample('D', on='timestamp').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last','volume': 'mean'}))
        day_df['timestamp'] = day_df.index

        start_point = random.choice(day_df['timestamp'])

        current_price = day_df.loc[start_point]['close']
        
        df_temp_next = day_df[day_df['timestamp'] > start_point]
        df_temp_next = df_temp_next[df_temp_next['timestamp'] < start_point + pd.DateOffset(days=WINDOW_AFTER)]
        max_price = max(df_temp_next['close'])

        if (max_price - current_price)/current_price > 0.5:
            print(market, start_point)
            print((max_price - current_price)/current_price)
            # continue

        df = df[df['timestamp'] <= start_point]
        day_df = day_df[day_df['timestamp'] <= start_point]
        num_idx = min([365, len(day_df)])

        model_price = Earth()
        scaler = preprocessing.MinMaxScaler(feature_range=(0,100))
        scaled_price = scaler.fit_transform(day_df['close'].iloc[-num_idx:].values.reshape(-1,1))

        arr_x = np.arange(num_idx)
        model_price.fit(arr_x,scaled_price.flatten())
        y_pred = model_price.predict(arr_x)
        basis_node = get_node_mars(model_price)
        last_node = int(basis_node[-1])
        last_slope = (y_pred[-1] - y_pred[last_node])/(arr_x[-1] - arr_x[last_node])

        adi = ta.volume.AccDistIndexIndicator(high=df['high'],low=df['low'],close=df['close'],volume=df['volume']).acc_dist_index()
        model_adi = Earth()
        scaler_adi = preprocessing.MinMaxScaler(feature_range=(0,1000))
        data_num_adi = min([2000,len(df)])
        scaled_adi = scaler_adi.fit_transform(df['close'].iloc[-data_num_adi:].values.reshape(-1,1))
        arr_x_adi = np.arange(data_num_adi)/24
        model_adi.fit(arr_x_adi,scaled_adi.flatten())

        y_pred_adi = model_adi.predict(arr_x_adi)
        basis_node = get_node_mars(model_adi)
        last_node = basis_node[-1]
        last_slope_adi = (y_pred_adi[-1] - model_adi.predict([last_node]))[0]/(arr_x[-1] - last_node)

        aroon = ta.trend.AroonIndicator(low=day_df['low'],high=day_df['high'])
        aroon_up = aroon.aroon_up()[-1]
        aroon_down = aroon.aroon_down()[-1]
        aroon_diff = aroon.aroon_indicator()[-1]

        macd = ta.trend.MACD(close=pd.Series(scaled_price.flatten()))
        macd_diff = macd.macd_diff().values[-1]
        macd_line = macd.macd().values[-1]
        macd_signal = macd.macd_signal().values[-1]

        stoch_oscillator = ta.momentum.StochasticOscillator(high=day_df['high'], low=day_df['low'], close=day_df['close'])
        stochastic = stoch_oscillator.stoch().values[-1]
        stochastic_signal = stoch_oscillator.stoch_signal().values[-1]
        delta_stochastic = stochastic - stochastic_signal

        rsi_indicator = ta.momentum.RSIIndicator(close=pd.Series(scaled_price.flatten()))
        rsi_val = rsi_indicator.rsi().values[-1]

        query = 'SELECT * FROM btcusdt ORDER BY timestamp ASC'
        res = execute_query(connection,query)
        df_btc = pd.DataFrame(res,columns=['timestamp', 'open','high', 'low','close','volume'])
        df_btc = df_btc.drop_duplicates()
        df_btc['timestamp'] = pd.to_datetime(df_btc['timestamp'], unit='ms')
        df_btc = df_btc[df_btc['timestamp'] <= start_point]
        
        day_df_btc = (df_btc.resample('D', on='timestamp').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last','volume': 'mean'}))
        day_df_btc['timestamp'] = day_df_btc.index

        alt_vs_btc = pd.DataFrame({'alt': day_df['close'].iloc[-num_idx:], 'btc': day_df_btc['close'].iloc[-num_idx:]})
        df_corr = alt_vs_btc.corr()
        coef_corr = df_corr['alt']['btc']

        model_btc = Earth()
        scaler_btc = preprocessing.MinMaxScaler(feature_range=(0,100))
        scaled_price_btc = scaler_btc.fit_transform(day_df_btc['close'].iloc[-num_idx:].values.reshape(-1,1))

        arr_x = np.arange(num_idx)
        model_btc.fit(arr_x,scaled_price_btc.flatten())

        y_pred_btc = model_btc.predict(arr_x)
        basis_node_btc = get_node_mars(model_btc)
        last_node_btc = int(basis_node_btc[-1])
        last_slope_btc = (y_pred_btc[-1] - y_pred_btc[last_node_btc])/(arr_x[-1] - arr_x[last_node_btc])

        arr_last_slope.append(last_slope)
        arr_last_slope_adi.append(last_slope_adi)
        arr_aroon_up.append(aroon_up)
        arr_aroon_down.append(aroon_down)
        arr_aroon_diff.append(aroon_diff)
        arr_macd_diff.append(macd_diff)
        arr_macd_line.append(macd_line)
        arr_macd_signal.append(macd_signal)
        arr_stochastic.append(stochastic)
        arr_stochastic_signal.append(stochastic_signal)
        arr_delta_stochastic.append(delta_stochastic)
        arr_rsi_val.append(rsi_val)
        arr_coef_corr.append(coef_corr)
        arr_last_slope_btc.append(last_slope_btc)
        arr_market.append(market)
        arr_start_date.append(start_point)
    except Exception as e:
        print('Error: ', e)
        arr_last_slope.append(np.nan)
        arr_last_slope_adi.append(np.nan)
        arr_aroon_up.append(np.nan)
        arr_aroon_down.append(np.nan)
        arr_aroon_diff.append(np.nan)
        arr_macd_diff.append(np.nan)
        arr_macd_line.append(np.nan)
        arr_macd_signal.append(np.nan)
        arr_stochastic.append(np.nan)
        arr_stochastic_signal.append(np.nan)
        arr_delta_stochastic.append(np.nan)
        arr_rsi_val.append(np.nan)
        arr_coef_corr.append(np.nan)
        arr_last_slope_btc.append(np.nan)
        arr_market.append(market)
        arr_start_date.append(start_point)
        continue

  0%|          | 0/1000 [00:00<?, ?it/s]

  2%|▏         | 18/1000 [01:53<1:49:35,  6.70s/it]

TIAUSDT 2023-11-25 00:00:00
0.706838253848095


  2%|▏         | 23/1000 [02:12<1:28:57,  5.46s/it]

OXTUSDT 2021-10-04 00:00:00
0.6040308239478366


...

 99%|█████████▉| 990/1000 [1:58:07<01:09,  6.99s/it]

TIAUSDT 2024-01-03 00:00:00
0.5581347677069155


100%|█████████▉| 999/1000 [1:58:36<00:03,  3.60s/it]

Error:  list index out of range


100%|██████████| 1000/1000 [1:58:41<00:00,  7.12s/it]


In [8]:
not_pump = pd.DataFrame({'market': arr_market,
                         'start': arr_start_date,
                         })
not_pump['last_slope'] = arr_last_slope
not_pump['last_slope_adi'] = arr_last_slope_adi
not_pump['aroon_up'] = arr_aroon_up
not_pump['aroon_down'] = arr_aroon_down
not_pump['aroon_diff'] = arr_aroon_diff
not_pump['macd_diff'] = arr_macd_diff
not_pump['macd_line'] = arr_macd_line
not_pump['macd_signal'] = arr_macd_signal
not_pump['stochastic'] = arr_stochastic
not_pump['stochastic_signal'] = arr_stochastic_signal
not_pump['delta_stochastic'] = arr_delta_stochastic
not_pump['rsi_val'] = arr_rsi_val
not_pump['coef_corr'] = arr_coef_corr
not_pump['last_slope_btc'] = arr_last_slope_btc
not_pump

,market,start,last_slope,last_slope_adi,aroon_up,aroon_down,aroon_diff,macd_diff,macd_line,macd_signal,stochastic,stochastic_signal,delta_stochastic,rsi_val,coef_corr,last_slope_btc
0,UMABTC,2021-08-06,-0.084034,0.313972,16.0,76.0,-60.0,0.155235,-0.719433,-0.874668,49.402390,44.938625,4.463766,44.525592,-0.400526,1.001631
1,LRCUSDT,2021-06-10,-0.995378,-0.438558,0.0,28.0,-28.0,0.235298,-5.115097,-5.350394,13.574993,18.007044,-4.432052,39.377034,0.920844,-0.211558
2,XVSBTC,2022-01-19,-0.044422,-0.468878,44.0,100.0,-56.0,-0.081013,-0.289282,-0.208269,15.017668,11.445796,3.571872,33.711375,0.211858,-0.470940
3,RSRUSDT,2022-02-12,-0.280737,0.237247,0.0,24.0,-24.0,0.385529,-1.170828,-1.556357,24.574669,40.264650,-15.689981,39.250531,0.618725,1.205207
4,MAGICUSDT,2023-08-17,-0.113555,-1.489297,56.0,100.0,-44.0,-0.705547,-1.364579,-0.659031,23.712068,19.343312,4.368756,24.017105,0.129985,-0.226496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,LAZIOBTC,2023-10-27,-0.086562,-1.127920,12.0,88.0,-76.0,-0.098610,-0.812158,-0.713548,20.104634,11.868320,8.236314,26.639754,-0.832667,2.791952
996,FIOBTC,2022-02-26,-0.155465,-0.337149,56.0,96.0,-40.0,-0.135897,-1.953652,-1.817755,8.080808,4.717240,3.363568,38.830719,0.004259,-1.380632
997,FLMBTC,2021-09-17,-0.127253,-0.089561,4.0,60.0,-56.0,-0.187308,-0.248029,-0.060722,26.150628,37.447699,-11.297071,42.744029,-0.220791,-0.274367
998,PLAUSDT,2021-11-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
not_pump_dropna = not_pump.dropna()
not_pump_dropna

,market,start,last_slope,last_slope_adi,aroon_up,aroon_down,aroon_diff,macd_diff,macd_line,macd_signal,stochastic,stochastic_signal,delta_stochastic,rsi_val,coef_corr,last_slope_btc
0,UMABTC,2021-08-06,-0.084034,0.313972,16.0,76.0,-60.0,0.155235,-0.719433,-0.874668,49.402390,44.938625,4.463766,44.525592,-0.400526,1.001631
1,LRCUSDT,2021-06-10,-0.995378,-0.438558,0.0,28.0,-28.0,0.235298,-5.115097,-5.350394,13.574993,18.007044,-4.432052,39.377034,0.920844,-0.211558
2,XVSBTC,2022-01-19,-0.044422,-0.468878,44.0,100.0,-56.0,-0.081013,-0.289282,-0.208269,15.017668,11.445796,3.571872,33.711375,0.211858,-0.470940
3,RSRUSDT,2022-02-12,-0.280737,0.237247,0.0,24.0,-24.0,0.385529,-1.170828,-1.556357,24.574669,40.264650,-15.689981,39.250531,0.618725,1.205207
4,MAGICUSDT,2023-08-17,-0.113555,-1.489297,56.0,100.0,-44.0,-0.705547,-1.364579,-0.659031,23.712068,19.343312,4.368756,24.017105,0.129985,-0.226496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,FLOWUSDT,2022-01-11,-0.020329,-1.870451,40.0,96.0,-56.0,-0.313641,-2.531093,-2.217452,20.264317,14.863762,5.400555,33.690728,-0.230688,-2.059122
995,LAZIOBTC,2023-10-27,-0.086562,-1.127920,12.0,88.0,-76.0,-0.098610,-0.812158,-0.713548,20.104634,11.868320,8.236314,26.639754,-0.832667,2.791952
996,FIOBTC,2022-02-26,-0.155465,-0.337149,56.0,96.0,-40.0,-0.135897,-1.953652,-1.817755,8.080808,4.717240,3.363568,38.830719,0.004259,-1.380632
997,FLMBTC,2021-09-17,-0.127253,-0.089561,4.0,60.0,-56.0,-0.187308,-0.248029,-0.060722,26.150628,37.447699,-11.297071,42.744029,-0.220791,-0.274367


In [10]:
old_df = pd.read_csv('data_not_pump.csv')
old_df = old_df.loc[:, ~old_df.columns.isin(['Unnamed: 0'])]
old_df

,market,start,last_slope,last_slope_adi,aroon_up,aroon_down,aroon_diff,macd_diff,macd_line,macd_signal,stochastic,stochastic_signal,delta_stochastic,rsi_val,coef_corr,last_slope_btc
0,MASKUSDT,2023-07-01,-0.259825,-0.725472,0.0,16.0,-16.0,-0.023778,-2.503912,-2.480134,40.082645,33.230028,6.852617,43.963420,0.653069,1.942003
1,CHRBTC,2022-01-12,-1.185157,-0.396954,60.0,8.0,52.0,-0.383539,3.070371,3.453910,46.379648,38.775392,7.604255,56.697912,0.343251,-0.518236
2,SSVUSDT,2023-11-24,0.519416,2.000122,100.0,16.0,84.0,0.751406,3.039644,2.288239,80.744681,68.427399,12.317282,70.363615,-0.338339,0.590438
3,BNBBTC,2024-02-01,0.033239,-0.776591,64.0,8.0,56.0,-0.859078,0.454805,1.313883,1.813785,7.077284,-5.263499,43.436891,-0.724380,-0.439969
4,UNFIUSDT,2022-12-28,-0.188445,0.129813,16.0,100.0,-84.0,-0.048580,-1.303688,-1.255108,15.000000,26.997657,-11.997657,35.416810,-0.008336,-0.133785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023,SEIUSDT,2024-01-31 00:00:00,1.040909,1.521766,44.0,68.0,-24.0,-1.508418,1.901538,3.409956,31.826401,38.049124,-6.222723,49.857619,0.749554,2.661619
1024,TUSDT,2022-08-17 00:00:00,0.062872,-0.551348,48.0,12.0,36.0,-0.044787,0.661372,0.706159,16.513761,31.941369,-15.427607,48.416397,0.924372,0.336691
1025,HIFIUSDT,2023-08-30 00:00:00,2.247303,-0.771860,80.0,48.0,32.0,1.176534,8.616437,7.439903,58.144439,67.474960,-9.330522,56.997903,-0.648747,-1.257968
1026,HIVEUSDT,2021-07-03 00:00:00,2.425457,0.513836,8.0,56.0,-48.0,2.231767,-2.786705,-5.018472,78.324896,85.175184,-6.850288,52.984398,0.736437,-0.190428


In [11]:
new_df = pd.concat([old_df, not_pump_dropna])
new_df

,market,start,last_slope,last_slope_adi,aroon_up,aroon_down,aroon_diff,macd_diff,macd_line,macd_signal,stochastic,stochastic_signal,delta_stochastic,rsi_val,coef_corr,last_slope_btc
0,MASKUSDT,2023-07-01,-0.259825,-0.725472,0.0,16.0,-16.0,-0.023778,-2.503912,-2.480134,40.082645,33.230028,6.852617,43.963420,0.653069,1.942003
1,CHRBTC,2022-01-12,-1.185157,-0.396954,60.0,8.0,52.0,-0.383539,3.070371,3.453910,46.379648,38.775392,7.604255,56.697912,0.343251,-0.518236
2,SSVUSDT,2023-11-24,0.519416,2.000122,100.0,16.0,84.0,0.751406,3.039644,2.288239,80.744681,68.427399,12.317282,70.363615,-0.338339,0.590438
3,BNBBTC,2024-02-01,0.033239,-0.776591,64.0,8.0,56.0,-0.859078,0.454805,1.313883,1.813785,7.077284,-5.263499,43.436891,-0.724380,-0.439969
4,UNFIUSDT,2022-12-28,-0.188445,0.129813,16.0,100.0,-84.0,-0.048580,-1.303688,-1.255108,15.000000,26.997657,-11.997657,35.416810,-0.008336,-0.133785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,FLOWUSDT,2022-01-11 00:00:00,-0.020329,-1.870451,40.0,96.0,-56.0,-0.313641,-2.531093,-2.217452,20.264317,14.863762,5.400555,33.690728,-0.230688,-2.059122
995,LAZIOBTC,2023-10-27 00:00:00,-0.086562,-1.127920,12.0,88.0,-76.0,-0.098610,-0.812158,-0.713548,20.104634,11.868320,8.236314,26.639754,-0.832667,2.791952
996,FIOBTC,2022-02-26 00:00:00,-0.155465,-0.337149,56.0,96.0,-40.0,-0.135897,-1.953652,-1.817755,8.080808,4.717240,3.363568,38.830719,0.004259,-1.380632
997,FLMBTC,2021-09-17 00:00:00,-0.127253,-0.089561,4.0,60.0,-56.0,-0.187308,-0.248029,-0.060722,26.150628,37.447699,-11.297071,42.744029,-0.220791,-0.274367


In [12]:
new_df.to_csv('data_not_pump.csv')